In [2]:
from heapq import merge
from operator import index
from sys import displayhook
import mysql.connector
from mysql.connector import Error
import numpy as np
import matplotlib.pyplot as plt
import mysql.connector as sql
import pandas as pd
from IPython.display import display
from sklearn.preprocessing import MinMaxScaler
import math

In [3]:
# Pengolahan Database
connection = mysql.connector.connect(host='localhost',
                                         database='cargo_aqil', # masukan nama database di kiri
                                         user='root',
                                         password='')
cursor = connection.cursor(dictionary=True)
querryRute = "SELECT sample.sample_id AS 'ID', origin AS'ID ASAL', destination AS 'ID TUJUAN', a.city_name AS 'ASAL', b.city_name AS 'TUJUAN', sample.distance AS 'JARAK', sample.price AS 'HARGA', sample.rate AS 'RATING', sample.duration AS 'DURASI' FROM sample JOIN city a ON sample.origin = a.city_id JOIN city b ON sample.destination = b.city_id;"
querryNodes = "SELECT city_id AS 'ID NODE', city_name AS 'NODE', lat, city.long FROM city"

In [4]:
def heuristicValues(nodes,tujuan):
    nodes["eDistance"] = np.nan
    p = nodes.loc[nodes.NODE == tujuan,'lat'].values
    q = nodes.loc[nodes.NODE == tujuan,'long'].values
    for node,lat,long in nodes[nodes.columns[1:4]].values:
        if node == tujuan:
            nodes.loc[nodes.NODE == tujuan,'eDistance'] = 0.0
        else:
            eDistance = math.sqrt((p-lat)**2+(q-long)**2)
            nodes.loc[nodes.NODE == node,'eDistance'] = eDistance*110.574

In [5]:
def heursiticGraph(nodes):
    hDict={}
    w1 = 1
    for row in nodes.values:
        node = row[1]
        e_distance = row[4]

        h_value = (e_distance*1/w1)
        hDict[node]=h_value
    return hDict

In [6]:
def normalisasiRute(rute,minParam):
    paramRute = rute.append(minParam,ignore_index=True)
    data = paramRute[paramRute.columns[5:9]]
    scaler = MinMaxScaler(feature_range=(5, 10))
    scaler.fit(data)
    normed =scaler.transform(data)
    rute[rute.columns[5:9]] = normed[:len(rute)]


In [7]:
def normalisasiHvalue(nodes,maxParam):
    paramNode = nodes.append(maxParam,ignore_index=True)
    data = paramNode[paramNode.columns[2:5]]
    scaler = MinMaxScaler(feature_range=(5, 10))
    scaler.fit(data)
    normed=scaler.transform(data)
    paramNode[paramNode.columns[2:5]]=normed
    nodes[nodes.columns[4]] = paramNode[paramNode.columns[4]].iloc[:len(nodes)]

In [8]:
def createGraph(rute):
    graph = {}
    w1,w2,w3,w4 = (1,1,1,1)
    for row in rute.values:
        origin_db = row[3]
        destination_db = row[4]
        distance_db = row[5]
        price_db = row[6]
        rating_db = row[7]
        duration_db = row[8]

        W = (distance_db*1/w1) + (price_db*1/w2) + (-rating_db*1/w3) + (duration_db*1/w4)

        if origin_db in graph:
            graph[origin_db][destination_db] = W
        else:
            graph[origin_db]={}
            graph[origin_db][destination_db] = W
    return graph

In [9]:
def greedy(asal,tujuan,graph):
    openList = {}
    closeList = []
    parent = {}
    labels = 'No Path Found'
    openList[asal]=0
    while tujuan not in closeList:
        bestNode = min(openList,key=openList.get)
        print("Bestnode :",bestNode)
        closeList.append(bestNode)
        if bestNode not in graph:
            closeList = labels
            break
        suksesor = graph[bestNode]
        for node,value in suksesor.items():
            #kalau node suksesor ada di open dan lebih besar valuenya diabaikan
            if node not in closeList:
                if node in openList:
                    if value<openList[node]:
                        openList[node]=value
                        parent[node]=bestNode
                elif node not in openList:
                    openList[node]=value
                    parent[node]=bestNode
        print(openList)
        del openList[bestNode]
        if tujuan in openList:
            bestNode = tujuan
            closeList.append(tujuan)
            break
    hasilSearch = closeList,parent
    return hasilSearch

In [10]:
def bestfirstSearch(asal,tujuan,graph):
    openList = {}
    closeList = []
    parent = {}
    labels = 'No Path Found'
    openList[asal]=0

    while openList != {}:
        if openList == {}:
            closeList = labels
        bestNode = min(openList,key=openList.get)
        print("Bestnode :",bestNode)
        closeList.append(bestNode)
        del openList[bestNode]
        if bestNode == tujuan:
            break
        if bestNode in graph:
            suksesor = graph[bestNode]
            for node,value in suksesor.items():
                if node not in closeList:
                    if node in openList:
                        if value<openList[node]:
                            openList[node]=value
                            parent[node]=bestNode
                    elif node not in openList:
                        openList[node]=value
                        parent[node]=bestNode         
            print(openList)
    hasilSearch = closeList,parent
    return hasilSearch

In [11]:
def dijkstra(asal,tujuan,graph):
    openList = {}
    closeList = []
    parent = {}
    labels = 'No Path Found'
    openList[asal]=0
    INF = 999999

    for node in graph.keys():
        for cNode in graph [node].keys():
            openList[cNode]=INF
            parent[cNode]=''
        openList[asal]=0
    print(openList)
    while openList != {}:
        if openList == {}:
            closeList = labels
            break
        bestNode = min(openList,key=openList.get)
        print("Bestnode :",bestNode)
        closeList.append(bestNode)
        if bestNode == tujuan:
            break
        if bestNode in graph:
            suksesor = graph[bestNode]
            for node,value in suksesor.items():
                if node in openList:
                    tempG = openList[bestNode]+graph[bestNode][node]
                    if tempG < openList[node]:
                        openList[node]=tempG
                        parent[node]=bestNode
        print(openList)
        del openList[bestNode]
    hasilSearch = closeList,parent
    return hasilSearch

In [111]:
def aStar(asal,tujuan,graph,hList):
    gList = {}
    openList = {}
    closeList = []
    parent = {}
    labels = 'No Path Found'
    openList[asal]=0

    listJalur = {}
    closeListB = []
    parentB = {}
    gList[asal] = 0
    while openList !={}:
        print("OPEN LIST :",openList)
        bestNode = min(openList,key=openList.get)
        print("Bestnode :",bestNode)
        closeList.append(bestNode)
        if bestNode in graph:
            suksesor = graph[bestNode]
            for node,value in suksesor.items():
                g = gList[bestNode] + graph[bestNode][node] #(g) baru  
                if node == tujuan:
                    print("ADA JALUR")
                    closeListB = closeList
                    parentB = parent
                    bestNodeB = tujuan
                    parentB[bestNodeB]=bestNode

                    fJalur = 0
                    jalur = []
                    while bestNodeB != asal:
                        print(bestNodeB)
                        print(parentB)
                        jalur.append(bestNodeB)
                        fJalur += graph[(parentB[bestNodeB])][bestNodeB] 
                        bestNodeB = parent[bestNodeB]
                    jalur.append(asal)
                    jalur.reverse()
                    print(jalur)
                    listJalur[fJalur]=jalur

                if node in openList:
                    parentLama = parent[node]
                    gLama =  gList[node]
                    if gLama > g:
                        gList[node]=g
                        openList[node]=g+hList[node]
                        parent[node]=bestNode
                elif node in closeList:
                    parentLama = parent[node]
                    gLama =  gList[node]
                    if gLama > g:
                        fBaru = g + hList[node]
                        parent[node]=bestNode
                        closeList[node]=fBaru
                        if node in graph:
                            suksesorNode = graph[node]
                            for cucu in suksesorNode.keys:
                                gCucu = g+suksesorNode[node]
                                if cucu in openList:
                                    gList = g+suksesorNode[node]
                                    openList[cucu]=g+suksesorNode[node]+hList[cucu]
                                    parent[cucu]=node
                                elif cucu in suksesor:
                                    if gCucu < g:
                                        gList = g+suksesorNode[node]
                                        openList[cucu]=gCucu+hList[cucu]
                                        parent[cucu]=node
                else:
                    gList[node]=g
                    openList[node]=g+hList[node]
                    parent[node]=bestNode
        del openList[bestNode]
    print("Jalur Yang Terdeteksi :",listJalur)
    hasilSearch = (listJalur[min(listJalur.keys())]),(min(listJalur.keys())-hList[listJalur[min(listJalur.keys())][-2]])
    return hasilSearch
    

In [185]:
def FloydWarshall(asal,tujuan):
    u = int(nodes.loc[nodes['NODE'] == asal]['ID NODE']-1)
    v = int(nodes.loc[nodes['NODE'] == tujuan]['ID NODE']-1)
    nV = len(nodes[nodes.columns[0]])
    matrix = []
    parent = []
    for k in range (nV):
        baris = []
        barisParent = []
        for i in range (nV):
            baris.append(999)
            barisParent.append(-1)
            if i == k:
                baris[i] = 0
                barisParent[i] = k

        matrix.append(baris)
        parent.append(barisParent)

    for row in rute.values:
        origin_db = row[1]
        destination_db = row[2]
        distance_db = row[5]
        price_db = row[6]
        rating_db = row[7]
        duration_db = row[8]
        W = (distance_db*1/w1) + (price_db*1/w2) + (-rating_db*1/w3) + (duration_db*1/w4)
        matrix[origin_db-1][destination_db-1]=W
        parent[origin_db-1][destination_db-1]=destination_db-1

    for row in parent:
        print(row)

    for k in range (nV):
        for i in range (nV):
            for j in range (nV):
                if matrix[i][j] > matrix[i][k] + matrix[k][j]:
                    matrix[i][j] = matrix[i][k] + matrix[k][j]
                    parent[i][j] = parent[i][k]

    for row in parent:
        print(row)

    if parent[u][v]==-1:
        hasil = "No Path"
        return hasil
    else:
        hasil = [u]
        bobotJalur = 0
        while u != v:
            bobotJalur += matrix[u][v]
            u = parent[u][v]
            hasil.append(u)

    jalur = []
    for idNode in hasil:
        node = nodes.loc[nodes['ID NODE'] == (idNode+1)]['NODE'].values[0]
        jalur.append(node)
    hasil=(jalur,bobotJalur)
    return hasil

In [71]:
def ekstrakJalur(closeList,parent,bestNode):
    print("PARENT :",parent)
    jalur = [] #sesuaikan parameter kedua ekstrakJalur dengan nama ini
    bobotJalur = 0 #sesuaikan parameter ketiga ekstrakJalur dengan nama var ini
    hasil = closeList
    if hasil != 'No Path Found':
        while bestNode != asal:
            jalur.append(bestNode)
            print(graph[parent[bestNode]][bestNode],":",graph[parent[bestNode]][bestNode])
            bobotJalur += graph[parent[bestNode]][bestNode]
            bestNode = parent[bestNode]
        jalur.append(asal)
        jalur.reverse()
        hasil = jalur,bobotJalur
    return hasil
    


In [188]:
rute = pd.read_sql(querryRute,connection)
nodes = pd.read_sql(querryNodes,connection)
asal = str(input("(asal)huruf pertama besar"))
tujuan = str(input("(tujuan)huruf pertama besar"))
algoritma = int(input("1. greedy/2. best first search/3. dijkstra/4.A star/5.FloydWarshall"))
minParam = {'ID': len(rute)+1, 'ASAL': 'Min Param', 'TUJUAN':'Min Param', 'JARAK': 0, 'HARGA':0, 'RATING':0, 'DURASI':0}
maxParam = {'ID NODE': len(nodes)+1, 'NODE': 'max_param', 'lat': 999, 'long':999, 'eDistance':max(rute[rute.columns[5]])}
normalisasiRute(rute,minParam) 
graph = createGraph(rute)
if algoritma == 1:
    hasil=ekstrakJalur(greedy(asal,tujuan,graph)[0],greedy(asal,tujuan,graph)[1],(greedy(asal,tujuan,graph)[0])[-1])
    print(hasil)
elif algoritma == 2:
    hasil=ekstrakJalur(bestfirstSearch(asal,tujuan,graph)[0],bestfirstSearch(asal,tujuan,graph)[1],(bestfirstSearch(asal,tujuan,graph)[0])[-1])
    print(hasil)
elif algoritma == 3:
    hasil=ekstrakJalur(dijkstra(asal,tujuan,graph)[0],dijkstra(asal,tujuan,graph)[1],(dijkstra(asal,tujuan,graph)[0])[-1])
    print(hasil)
elif algoritma == 4:
    heuristicValues(nodes,tujuan)
    normalisasiHvalue(nodes,maxParam)
    hList=heursiticGraph(nodes)
    hasil = aStar(asal,tujuan,graph,hList)
    print(hasil)
elif algoritma == 5:
    hasil=FloydWarshall(asal,tujuan)
    print(hasil)
    
    


(asal)huruf pertama besarJakarta
(tujuan)huruf pertama besarJayapura
1. greedy/2. best first search/3. dijkstra/4.A star/5.FloydWarshall5
[0, -1, 2, 3, 4, 5, 6, 7, 8, 9, 10, -1, -1, 13, -1, -1, -1, -1, -1, -1, -1, -1]
[-1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[-1, -1, 2, -1, -1, -1, -1, -1, -1, -1, 10, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[-1, -1, -1, 3, -1, 5, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[-1, -1, -1, -1, 4, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[-1, 1, -1, -1, -1, 5, -1, 7, 8, -1, -1, -1, -1, 13, -1, -1, 16, -1, -1, -1, 20, 21]
[-1, -1, -1, -1, -1, -1, 6, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[-1, -1, -1, -1, -1, -1, -1, 7, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[-1, -1, -1, -1, -1, -1, -1, -1, 8, -1, 10, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[-1, -1, -1, -1, -1, -1, -1, -1, -1, 9, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1